In [1]:
%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import json
import yaml
from copy import deepcopy
from multiprocessing import Pool

import pandas as pd
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits

import gPhoton as gp

In [4]:
import vasca.utils as vutils

In [5]:
# Event sky locations
locs = {
    "PS1-10jh": {
        "ra": 242.368075,
        "dec": 53.673492,
        "trans_type": "TDE",
        "pub_date": "2012",
    },
    "PS1-11af": {
        "ra": 149.36175,
        "dec": 3.23361,
        "trans_type": "TDE",
        "pub_date": "2013",
    },
    "D3-13": {
        "ra": 214.8741,
        "dec": 52.8684,
        "trans_type": "TDE",
        "pub_date": "2006",
    },
    "D1-9": {
        "ra": 36.3207,
        "dec": 4.5497,
        "trans_type": "TDE",
        "pub_date": "2008",
    },
    "D23H-1": {
        "ra": 352.99804167,
        "dec": 0.28738056,
        "trans_type": "TDE",
        "pub_date": "2009",
    },
    "PS1-13arp": {
        "ra": 184.604295,
        "dec": 46.616852,
        "trans_type": "SN2p",
        "pub_date": "2008",
    },
    "SN-2010aq": {
        "ra": 150.5405875,
        "dec": 1.23359444,
        "trans_type": "SN2p",
        "pub_date": "2010",
    },
    "test": {
        "ra": 176.919525856024,
        "dec": 0.255696872807351,
        "trans_type": "test",
        "pub_date": "0",
    },
}

df_locs = pd.DataFrame.from_dict(locs)
df_locs

,PS1-10jh,PS1-11af,D3-13,D1-9,D23H-1,PS1-13arp,SN-2010aq,test
ra,242.368075,149.36175,214.8741,36.3207,352.998042,184.604295,150.540587,176.919526
dec,53.673492,3.23361,52.8684,4.5497,0.287381,46.616852,1.233594,0.255697
trans_type,TDE,TDE,TDE,TDE,TDE,SN2p,SN2p,test
pub_date,2012,2013,2006,2008,2009,2008,2010,0


In [11]:
out_dir = "./out"
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

In [6]:
transient = "test"
band = "NUV"
ra = df_locs[transient].ra
dec = df_locs[transient].dec

In [12]:
@vutils.timeit
def gfind_and_save(transient):
    band = "NUV"
    ra = df_locs[transient].ra
    dec = df_locs[transient].dec
    
    gfind_res = gp.gFind(
        band=band, skypos=[ra, dec], maxgap=100, retries=3, quiet=True,
    )
    
    with open(f"{out_dir}/{transient}_gFind_res.json", "w") as f:
        # convert to JSON-compatible types
        out_data = deepcopy(gfind_res)
        for key in out_data[band].keys():
            if isinstance(out_data[band][key], np.ndarray):
                out_data[band][key] = out_data[band][key].tolist()
        # write        
        json.dump(out_data, f, indent=4)

In [9]:
gfind_and_save("SN 2010aq")

func: gfind_and_save , args: [('SN 2010aq',)] took:     231.05 sec
